# Import Libraries

In [169]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Import Data

In [170]:
data = pd.read_excel('Dataset.xlsx') 

In [171]:
data

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,GSW vs. PHX,10/24/2023,L,240,104,36,101,35.6,10,...,78.6,18,31,49,19,11,6,11,23,-4
1,PHX,PHX @ GSW,10/24/2023,W,240,108,42,95,44.2,11,...,76.5,17,43,60,23,5,7,19,22,4
2,LAL,LAL @ DEN,10/24/2023,L,240,107,41,90,45.6,10,...,75,13,31,44,23,5,4,12,18,-12
3,DEN,DEN vs. LAL,10/24/2023,W,240,119,48,91,52.7,14,...,75,9,33,42,29,9,6,12,15,12
4,MEM,MEM vs. NOP,10/25/2023,L,240,104,38,91,41.8,12,...,80,8,29,37,23,8,7,13,19,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,CLE,CLE vs. CHA,04/14/2024,L,240,110,44,93,47.3,16,...,66.7,12,31,43,30,4,1,14,8,-10
2456,NYK,NYK vs. CHI,04/14/2024,W,265,120,46,91,50.5,12,...,76.2,16,37,53,27,7,6,21,17,1
2457,LAC,LAC vs. HOU,04/14/2024,L,240,105,41,104,39.4,8,...,88.2,16,35,51,23,13,8,12,14,-11
2458,LAL,LAL @ NOP,04/14/2024,W,240,124,48,87,55.2,11,...,94.4,7,32,39,32,12,3,15,13,16


# Check Data

In [172]:
print("Missing Data Summary:")
print(data.isnull().sum())

Missing Data Summary:
Team         0
Match Up     0
Game Date    0
W/L          0
MIN          0
PTS          0
FGM          0
FGA          0
FG%          0
3PM          0
3PA          0
3P%          0
FTM          0
FTA          0
FT%          0
OREB         0
DREB         0
REB          0
AST          0
STL          0
BLK          0
TOV          0
PF           0
+/-          0
dtype: int64


# Feature Engineering
1. Check Correlation Matrix to see variable correlations;
    - Variables chosen (above 0.5 correlation with PTS)
        - FGM
        - FG%
        - +/-
        - AST
        - 3PM

In [173]:
numerical_columns = ['PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-'] 
corr_matrix = data[numerical_columns].corr()
print(corr_matrix['PTS'].sort_values(ascending=False))


PTS    1.000000
FGM    0.864809
FG%    0.744538
+/-    0.614423
AST    0.599123
3PM    0.522214
FGA    0.292205
PF     0.159309
3PA    0.144216
REB    0.122808
BLK    0.097010
STL    0.045898
TOV   -0.187124
Name: PTS, dtype: float64


# Model

## Construct new data frame to for model construction
- Select variables: 
    - Team
    - Game Date
    - PTS
    - FGM
    - FG%
    - +/-
    - AST
    - 3PM
    - Home_Game

- Group Team togethers
- Sort time in ascending order

Questions to use along the way: What to do with W/L, do I use W/L as the binary classifictaion or use the regression at first?

- What's the relationship between W/L and PTS?
- Can I predict PTS first and the W/L
- or make PTS as one of the feature?
- Do each team have their own function class?



In [174]:
df = data[['Team', 'Match Up', 'Game Date', 'W/L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-']]
df['Home_Game'] = df['Match Up'].apply(lambda x: 1 if "vs." in x else 0)
df = df.drop(columns = 'Match Up')
df['Game Date'] = pd.to_datetime(df['Game Date'], format='%m/%d/%Y')
df = df.sort_values(by=['Team', 'Game Date']).reset_index(drop=True)
df['W/L'] = df['W/L'].map({'W': 1, 'L': 0})
df

,Team,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Home_Game
0,ATL,2023-10-25,0,240,110,39,93,41.9,5,29,...,12,30,42,24,12,1,12,19,-6,0
1,ATL,2023-10-27,0,240,120,42,87,48.3,12,32,...,9,35,44,28,7,6,14,20,-6,1
2,ATL,2023-10-29,1,240,127,47,93,50.5,15,37,...,13,33,46,32,15,2,17,17,17,0
3,ATL,2023-10-30,1,240,127,48,86,55.8,14,30,...,4,32,36,28,6,7,11,12,14,1
4,ATL,2023-11-01,1,240,130,46,92,50.0,9,32,...,14,43,57,26,8,3,21,16,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,WAS,2024-04-05,0,240,102,38,88,43.2,5,33,...,12,33,45,24,9,5,10,21,-6,1
2456,WAS,2024-04-07,0,240,122,42,89,47.2,16,40,...,8,35,43,29,9,11,15,25,-8,0
2457,WAS,2024-04-09,0,240,121,44,89,49.4,20,43,...,7,34,41,26,5,4,13,22,-9,0
2458,WAS,2024-04-12,0,240,127,45,86,52.3,18,43,...,4,35,39,32,6,5,12,22,-2,1


In [175]:
X_train.iloc[1]

Home_Game       0
MIN           240
PTS           102
FGM            37
FGA            87
FG%          42.5
3PM            11
3PA            34
3P%          32.4
FTM            17
FTA            23
FT%          73.9
OREB           12
DREB           30
REB            42
AST            23
STL             4
BLK             4
TOV            12
PF             19
+/-           -12
Name: 59, dtype: object

In [176]:
df_model = df[['Team','Home_Game', 'W/L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-']]


train_data_list = []
test_data_list = []

df_model = df_model.sort_values(by='Team')  # Sort by team and game date
for team, group in df_model.groupby('Team'):
    train_size = int(len(group) * 0.7)
    train_data_list.append(group.iloc[:train_size])  # First 70% for training
    test_data_list.append(group.iloc[train_size:])  # Remaining 30% for testing

# Concatenate the training and testing data for all teams
train_data = pd.concat(train_data_list)
test_data = pd.concat(test_data_list)

X_train = train_data.drop(columns=['W/L', 'Team'])
y_train = train_data['W/L']
X_test = test_data.drop(columns=['W/L', 'Team'])
y_test= test_data['W/L']


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the Logistic Regression model
lasso_model = LogisticRegression(penalty='l1', solver='saga', random_state=42, max_iter=10000)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(lasso_model, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_scaled, y_train)

# Best model
best_lasso_model = grid.best_estimator_

# Make predictions
y_pred = best_lasso_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

# Feature importance
lasso_coefficients = best_lasso_model.coef_.flatten()
feature_importance = pd.DataFrame({'Feature': train_data.drop(columns=['W/L', 'Team']).columns, 'Coefficient': lasso_coefficients})
important_features = feature_importance[feature_importance['Coefficient'] != 0]

# Display results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report_str)
print("\nImportant Features:")
print(important_features)


ValueError: could not convert string to float: '-'

In [177]:
# Check for non-numeric values in the dataset
non_numeric_values = X_train.applymap(lambda x: isinstance(x, str) and not x.isnumeric()).any()
print("Columns with non-numeric values:")
print(non_numeric_values[non_numeric_values].index.tolist())


Columns with non-numeric values:
['FT%']


/var/folders/s8/_l25g4l10y7725mc4fmr9mpm0000gn/T/ipykernel_14984/3803749129.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_numeric_values = X_train.applymap(lambda x: isinstance(x, str) and not x.isnumeric()).any()


In [178]:
df_model['FT%']

0       81.8
59      73.9
58      71.4
57        87
56      88.2
        ... 
2401    76.9
2400    79.2
2399    64.5
2396      71
2459     100
Name: FT%, Length: 2460, dtype: object

In [179]:
unique_values = df['FT%'].unique()
print("Unique values in 'FT%' column:", unique_values)

Unique values in 'FT%' column: [81.8 80 94.4 90.6 79.2 84.4 76.7 83.3 92.3 76.9 82.1 93.1 70 63.2 91.7
 86.2 76.2 90.5 73.7 88 81.5 71.4 85.7 84 64.9 93.9 86.7 58.6 73.8 73.9
 72.7 90.9 82.4 66.7 77.8 88.9 77.3 90 75.7 82.8 70.8 88.2 87 78.3 75 74.1
 79.3 58.8 60 78.6 62.5 68 92.6 71.9 73.1 64.7 84.6 65 73.3 74.2 95.2 76.5
 69.2 81.3 62.8 63 64.5 50 61.5 84.8 93.8 87.5 59.5 94.7 65.4 93.3 57.1
 56.7 72.2 85.2 77.4 68.4 63.6 67.9 70.4 78.8 96.4 84.2 78.9 64.3 73 86.1
 100 85 52.6 69 96.8 76 92 95 92.9 70.6 91.9 64 89.5 '-' 72 74.3 86.4 94.1
 82.6 60.7 68.8 89.3 78.1 68.2 69.6 95.5 83.9 55.6 58.3 91.3 89.7 87.1 81
 72.4 81.6 75.8 88.5 79.1 71 80.8 95.7 67.7 57.9 37.5 61.3 65.7 47.4 70.7
 82.9 74.4 41.2 53.6 42.9 65.2 61.1 45.5 56.5 75.9 65.5 54.5 33.3 77.1
 59.4 80.6 69.7 73.5 90.3 96 35.7 65.6 60.9 56.3 61.9 96.2 78.4 81.1 88.4
 56 68.3 59.1 95.8 79.4 69.4 48 88.6 96.3 52 53.3 70.3 43.8 94.3 52.4 77.5
 75.6 53.8 87.9 67.6 57.6 80.5 44.4 85.4 97.1 82.2 85.3 72.1 93.5 38.9
 81.4 55 52.2 5

In [180]:
# Clean the 'FT%' column specifically
X_train['FT%'] = X_train['FT%'].replace('-', pd.NA)
X_test['FT%'] = X_test['FT%'].replace('-', pd.NA)

# Convert to numeric, coercing any remaining invalid values to NaN
X_train['FT%'] = pd.to_numeric(X_train['FT%'], errors='coerce')
X_test['FT%'] = pd.to_numeric(X_test['FT%'], errors='coerce')

# Fill NaN values with the mean of the column (calculated from training data)
X_train['FT%'].fillna(X_train['FT%'].mean(), inplace=True)
X_test['FT%'].fillna(X_train['FT%'].mean(), inplace=True)  # Use training mean fo

In [182]:
df_model = df[['Team','Home_Game', 'W/L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-']]


train_data_list = []
test_data_list = []

df_model = df_model.sort_values(by='Team')  # Sort by team and game date
for team, group in df_model.groupby('Team'):
    train_size = int(len(group) * 0.7)
    train_data_list.append(group.iloc[:train_size])  # First 70% for training
    test_data_list.append(group.iloc[train_size:])  # Remaining 30% for testing

# Concatenate the training and testing data for all teams
train_data = pd.concat(train_data_list)
test_data = pd.concat(test_data_list)

X_train = train_data.drop(columns=['W/L', 'Team'])
y_train = train_data['W/L']
X_test = test_data.drop(columns=['W/L', 'Team'])
y_test= test_data['W/L']

# Clean the 'FT%' column specifically
X_train['FT%'] = X_train['FT%'].replace('-', pd.NA)
X_test['FT%'] = X_test['FT%'].replace('-', pd.NA)

# Convert to numeric, coercing any remaining invalid values to NaN
X_train['FT%'] = pd.to_numeric(X_train['FT%'], errors='coerce')
X_test['FT%'] = pd.to_numeric(X_test['FT%'], errors='coerce')

# Fill NaN values with the mean of the column (calculated from training data)
X_train['FT%'].fillna(X_train['FT%'].mean(), inplace=True)
X_test['FT%'].fillna(X_train['FT%'].mean(), inplace=True)  # Use training mean fo

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the Logistic Regression model
lasso_model = LogisticRegression(penalty='l1', solver='saga', random_state=42, max_iter=10000)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(lasso_model, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_scaled, y_train)

# Best model
best_lasso_model = grid.best_estimator_

# Make predictions
y_pred = best_lasso_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

# Feature importance
lasso_coefficients = best_lasso_model.coef_.flatten()
feature_importance = pd.DataFrame({'Feature': train_data.drop(columns=['W/L', 'Team']).columns, 'Coefficient': lasso_coefficients})
important_features = feature_importance[feature_importance['Coefficient'] != 0]

# Display results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report_str)
print("\nImportant Features:")
print(important_features)


Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       370
           1       1.00      1.00      1.00       380

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Important Features:
   Feature  Coefficient
20     +/-     3.008799
